In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import plotly.express as px

In [ ]:
baseline_submissions = pd.read_csv('../results/baseline_submission.csv')
baseline_submissions_1 = pd.read_csv('../results/baseline_submission_1.csv')

In [ ]:
all_events = pd.read_csv('../data/raw/train_events.csv')

In [ ]:
def prepare_series(predicted_events):
  train_series = pd.DataFrame()
  for series in predicted_events['series_id'].drop_duplicates():
    train_series = pd.concat([train_series, pd.read_parquet("../data/raw/train_series.parquet", filters=[('series_id','=',series)])], axis=0)
  return train_series

In [ ]:
series = prepare_series(predicted_events=baseline_submissions)
series_1 = prepare_series(predicted_events=baseline_submissions_1)

In [ ]:
def print_graph(time_series, predicted_events, true_events):
  for series_id in predicted_events['series_id'].drop_duplicates():
    current_series = time_series[time_series['series_id'] == series_id]
    
    fig = px.line(current_series, x='timestamp', y='anglez', title=f'Schlafdaten für {series_id}')
    for index, row in predicted_events[predicted_events['series_id'] == series_id].dropna().iterrows():
        color = 'green' if row.event == 'onset' else 'red' 
        fig.add_vline(x=row.timestamp, line_width=3, line_color=color)

    for index, row in true_events[true_events['series_id'] == series_id].dropna().iterrows():
        color = 'magenta' if row.event == 'onset' else 'goldenrod' 
        fig.add_vline(x=row.timestamp, line_width=3, line_dash='dash', line_color=color)

    # fig.update_layout(legend=dict(
    #   yanchor="top",
    #   y=0.99,
    #   xanchor="left",
    #   x=0.99
    # ))

    fig.show()

In [ ]:
print_graph(series, baseline_submissions, all_events)

In [ ]:
print_graph(series_1, baseline_submissions_1, all_events)
